In [ ]:
#  Less Data when using the query string below

##  s = searchContext.query('query_string', query='event.module:zeek')

## Unable to find Empire Beacon which is Destination.IP 192.168.120.158

In [23]:
#Elasticsearch connector
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import eland as ed
#Data manipulation
import pandas as pd
from pandas.io.json import json_normalize
import json
import re
import eland as ed
import numpy as np
import matplotlib.pyplot as plt
import math
from datetime import datetime
# Disable warnings"
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [24]:
es = Elasticsearch(['https://192.168.1.181:9200'],                  
ca_certs=False,verify_certs=False, http_auth=('jupyter','password'))

# defining the search statement to get all records in an index
searchContext = Search(using=es, index='*:so-*', doc_type='doc')
es.info()

ObjectApiResponse({'name': 'voodoo-onion', 'cluster_name': 'voodoo-onion', 'cluster_uuid': 'dP1VycCSTQ2EPeCVxQ25GA', 'version': {'number': '7.16.2', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '2b937c44140b6559905130a8650c64dbd0879cfb', 'build_date': '2021-12-18T19:42:46.604893745Z', 'build_snapshot': False, 'lucene_version': '8.10.1', 'minimum_wire_compatibility_version': '6.8.0', 'minimum_index_compatibility_version': '6.0.0-beta1'}, 'tagline': 'You Know, for Search'})

In [25]:
query_since_ts = 0
body_filter = '{"query":{"bool":{"filter":[{"range":{@timestamp":{"gt":' + str(query_since_ts) + '}}}]}},"sort":[{"@timstamp":{"order":"asc"}}]}'
#body_filter = '{"query":{"@timestamp":{}},"size=10000":[{"@timestamp":{"order":"desc"}}]}'

In [27]:
#s = searchContext.query('query_string', query='message:("ts" AND "id.orig_h" AND "id.resp_h" AND "host" AND "id.resp_p" AND "method")')
 
#s = searchContext.query('query_string', query='destination.port:*')

#s = searchContext.query('query_string', query='*')

#s = searchContext.query('query_string', query='(destination.port:"*" OR http.virtual_host:"*")')

#s = searchContext.query('query_string', query='http.virtual_host:*')

#s = searchContext.query('query_string', query='http.method:*')

s = searchContext.query('query_string', query='event.module:zeek')

In [28]:
pd.set_option("display.max_rows", 100)
pd.set_option("max_colwidth", 600)
pd.set_option("display.max_columns", None)


#base response query
#response = s.execute()
#if response.success():
#  df = pd.DataFrame((d.to_dict() for d in s.scan()))
#json_struct = json.loads(df.to_json(orient="records"))
#df_flat = pd.io.json.json_normalize(json_struct)
#df_flat.head(20)

#response = s.execute()
#if response.success():
    #df = pd.DataFrame(([d['@timestamp'], d['source']['ip'], d['destination']['ip'], d['http']['virtual_host'], d['destination']['port'], d['http']['method']] for d in s))
#df.columns=['Timestamp','Source.ip','Destination.ip','Host','Destination.port','Method']    
#df.astype({'Destination.Port': 'int64'})
#display(df)
#df.info()

In [29]:
response = s.execute()
df = pd.DataFrame(columns=['ts', 'source.ip','destination.ip','virtual_host','destination.port'])
if response.success():
    for d in s[:10000]:
        try:
            df = df.append({'ts' : d['@timestamp'], 'source.ip' : d['source']['ip'], 'destination.ip' : d['destination']['ip'], 'virtual_host': d['http']['virtual_host'], 'destination.port': d['destination']['port'], 'http.method': d['http']['method']}, ignore_index=True)
        except KeyError:
            pass
            
#response = s.execute()
#if response.success():
#    df = pd.DataFrame(([d['@timestamp'], d['source']['ip'], d['destination']['ip'], d['http']['virtual_host'], d['destination']['port'], d['http']['method']] for d in s))
#df.columns=['Timestamp','Source.ip','Destination.ip','Host','Destination.port','Method']    
#df.astype({'Destination.Port': 'int64'})

display(df)

df.info(50)

,ts,source.ip,destination.ip,virtual_host,destination.port,http.method
0,2021-12-29T22:57:46.131Z,192.168.128.25,72.21.81.240,ctldl.windowsupdate.com,80,GET
1,2021-12-29T21:53:16.471Z,192.168.128.30,34.107.221.82,detectportal.firefox.com,80,GET
2,2021-12-29T21:54:16.167Z,192.168.128.50,72.21.91.29,ocsp.digicert.com,80,GET
3,2021-12-29T21:54:19.107Z,192.168.1.154,72.21.81.240,ctldl.windowsupdate.com,80,GET
4,2021-12-29T21:53:39.093Z,192.168.1.154,72.21.81.240,ctldl.windowsupdate.com,80,GET
...,...,...,...,...,...,...
218,2021-12-29T22:20:54.041Z,192.168.1.154,23.66.101.177,download.windowsupdate.com,80,GET
219,2021-12-29T22:20:52.575Z,192.168.1.154,23.66.101.177,download.windowsupdate.com,80,GET
220,2021-12-29T22:20:53.202Z,192.168.128.15,23.66.101.177,download.windowsupdate.com,80,GET
221,2021-12-29T22:20:54.485Z,192.168.128.15,23.66.101.177,download.windowsupdate.com,80,GET


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 223 entries, 0 to 222
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   ts                223 non-null    object
 1   source.ip         223 non-null    object
 2   destination.ip    223 non-null    object
 3   virtual_host      223 non-null    object
 4   destination.port  223 non-null    object
 5   http.method       223 non-null    object
dtypes: object(6)
memory usage: 10.6+ KB


In [32]:
df['ts'] = pd.to_datetime(df['ts'],infer_datetime_format=True)
df.head(5)

,ts,source.ip,destination.ip,virtual_host,destination.port,http.method
0,2021-12-29 22:20:54.410000+00:00,192.168.128.15,23.66.101.177,download.windowsupdate.com,80,GET
1,2021-12-29 22:20:54.545000+00:00,192.168.1.154,23.66.101.177,download.windowsupdate.com,80,GET
2,2021-12-29 22:20:53.313000+00:00,192.168.128.15,23.66.101.177,download.windowsupdate.com,80,GET
3,2021-12-29 22:20:53.250000+00:00,192.168.128.15,23.66.101.177,download.windowsupdate.com,80,GET
4,2021-12-29 22:20:54.212000+00:00,192.168.128.15,23.66.101.177,download.windowsupdate.com,80,GET


In [33]:
# assign field/column names to variables
f_timestamp = 'ts'
f_src_ip = 'source.ip'
f_dst_ip = 'destination.ip'
f_dst_host = 'virtual_host'
f_dst_port = 'destination.port'
f_http_method = 'http.method'
f_delimiter = '\t'

columns_to_filter = [f_timestamp, f_src_ip, f_dst_ip, f_dst_host, f_dst_port, f_http_method]
columns_to_groupby = [f_src_ip, f_dst_ip, f_dst_host, f_dst_port, f_http_method]
# columns to display after the analysis
columns_to_display = ['tsScore','conn_count',f_src_ip,f_dst_ip,f_dst_host,f_http_method, f_dst_port,'deltas']

In [34]:
# If you have a large dateset, using groupby and aggregate(list) might be slow. Consider using dask
df = df.groupby(columns_to_groupby).agg(list)
df.head(50)

ts
source.ip      destination.ip  virtual_host                       destination.port http.method                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         
192.168.1.154  156.112.111.142 ocsp.disa.mil                      80               GET                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               [2021-12-29 20:54:43.364000+00:00]
               184.86.177.242  tile-service.weather.microsoft.com 80               GET                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             [2021-12-29 21:55:44.998000+00:00, 2021-12-29 21:53:22.128000+00:00]
               209.197.3.8     ctldl.windowsupdate.com            80               GET                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               [2021-12-29 21:47:30.557000+00:00]
               23.49.5.177     ctldl.windowsupdate.com            80               GET                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               [2021-12-29 21:02:14.085000+00:00]
               23.66.101.177   ctldl.windowsupdate.com            80               GET                                                                                                                                                                                                                                                                                                                                                                                                                                               

In [10]:
df.reset_index(inplace=True)
df.head(5)

,source.ip,destination.ip,virtual_host,destination.port,http.method,ts
0,192.168.1.154,104.18.24.243,ocsp.msocsp.com,80,GET,[2021-12-29 21:55:51.058000+00:00]
1,192.168.1.154,104.80.74.140,x1.c.lencr.org,80,GET,[2021-12-29 21:59:27.760000+00:00]
2,192.168.1.154,13.107.4.50,ctldl.windowsupdate.com,80,GET,[2021-12-29 20:55:47.734000+00:00]
3,192.168.1.154,13.107.4.52,www.msftconnecttest.com,80,GET,[2021-12-29 21:53:18.923000+00:00]
4,192.168.1.154,156.112.111.142,ocsp.disa.mil,80,GET,"[2021-12-29 20:54:43.364000+00:00, 2021-12-29 20:54:43.446000+00:00]"


In [11]:
# create a new column 'conn_count', and for each row in the 'ts' column, apply a function and assign the returned value to the 'conn_count' column
df['conn_count'] = df[f_timestamp].apply(lambda x: len(x))
df.head(2)

,source.ip,destination.ip,virtual_host,destination.port,http.method,ts,conn_count
0,192.168.1.154,104.18.24.243,ocsp.msocsp.com,80,GET,[2021-12-29 21:55:51.058000+00:00],1
1,192.168.1.154,104.80.74.140,x1.c.lencr.org,80,GET,[2021-12-29 21:59:27.760000+00:00],1


In [19]:
#Filter out traffic where the connection count is quite small
df = df.loc[df['conn_count'] > 2]
df.shape

(2, 20)

In [20]:
df[f_timestamp] = df[f_timestamp].apply(lambda x: sorted(x))
df.head()

,source.ip,destination.ip,virtual_host,destination.port,http.method,ts,conn_count,deltas,tsLow,tsMid,tsHigh,tsBowleyNum,tsBowleyDen,tsSkew,tsMadm,tsConnDiv,tsSkewScore,tsMadmScore,tsConnCountScore,tsScore
0,192.168.1.154,8.240.196.126,download.windowsupdate.com,80,GET,"[2021-12-29 22:03:21.246000+00:00, 2021-12-29 22:03:21.305000+00:00, 2021-12-29 22:03:21.331000+00:00, 2021-12-29 22:03:21.405000+00:00, 2021-12-29 22:03:21.444000+00:00, 2021-12-29 22:03:21.486000+00:00, 2021-12-29 22:03:21.523000+00:00, 2021-12-29 22:03:21.568000+00:00, 2021-12-29 22:03:21.588000+00:00, 2021-12-29 22:03:21.615000+00:00, 2021-12-29 22:03:21.670000+00:00, 2021-12-29 22:03:21.697000+00:00, 2021-12-29 22:03:21.750000+00:00, 2021-12-29 22:03:21.812000+00:00, 2021-12-29 22:03:21.818000+00:00, 2021-12-29 22:03:21.890000+00:00, 2021-12-29 22:03:23.185000+00:00, 2021-12-29 22:03:...",158,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...]",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.9,1.0,1.0,1.0,1.0
1,192.168.128.50,8.240.196.126,download.windowsupdate.com,80,GET,"[2021-12-29 22:03:21.246000+00:00, 2021-12-29 22:03:21.305000+00:00, 2021-12-29 22:03:21.331000+00:00, 2021-12-29 22:03:21.405000+00:00, 2021-12-29 22:03:21.444000+00:00, 2021-12-29 22:03:21.486000+00:00, 2021-12-29 22:03:21.523000+00:00, 2021-12-29 22:03:21.534000+00:00, 2021-12-29 22:03:21.588000+00:00, 2021-12-29 22:03:21.614000+00:00, 2021-12-29 22:03:21.670000+00:00, 2021-12-29 22:03:21.697000+00:00, 2021-12-29 22:03:21.750000+00:00, 2021-12-29 22:03:21.812000+00:00, 2021-12-29 22:03:21.817000+00:00, 2021-12-29 22:03:21.890000+00:00, 2021-12-29 22:03:23.185000+00:00, 2021-12-29 22:03:...",158,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...]",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.9,1.0,1.0,1.0,1.0


In [21]:
# Convert list into a Series object, get time delta, convert the result back into a list and assign it to the 'deltas' column
df['deltas'] = df[f_timestamp].apply(lambda x: pd.Series(x).diff().dt.seconds.dropna().tolist())
df.head(2)

,source.ip,destination.ip,virtual_host,destination.port,http.method,ts,conn_count,deltas,tsLow,tsMid,tsHigh,tsBowleyNum,tsBowleyDen,tsSkew,tsMadm,tsConnDiv,tsSkewScore,tsMadmScore,tsConnCountScore,tsScore
0,192.168.1.154,8.240.196.126,download.windowsupdate.com,80,GET,"[2021-12-29 22:03:21.246000+00:00, 2021-12-29 22:03:21.305000+00:00, 2021-12-29 22:03:21.331000+00:00, 2021-12-29 22:03:21.405000+00:00, 2021-12-29 22:03:21.444000+00:00, 2021-12-29 22:03:21.486000+00:00, 2021-12-29 22:03:21.523000+00:00, 2021-12-29 22:03:21.568000+00:00, 2021-12-29 22:03:21.588000+00:00, 2021-12-29 22:03:21.615000+00:00, 2021-12-29 22:03:21.670000+00:00, 2021-12-29 22:03:21.697000+00:00, 2021-12-29 22:03:21.750000+00:00, 2021-12-29 22:03:21.812000+00:00, 2021-12-29 22:03:21.818000+00:00, 2021-12-29 22:03:21.890000+00:00, 2021-12-29 22:03:23.185000+00:00, 2021-12-29 22:03:...",158,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...]",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.9,1.0,1.0,1.0,1.0
1,192.168.128.50,8.240.196.126,download.windowsupdate.com,80,GET,"[2021-12-29 22:03:21.246000+00:00, 2021-12-29 22:03:21.305000+00:00, 2021-12-29 22:03:21.331000+00:00, 2021-12-29 22:03:21.405000+00:00, 2021-12-29 22:03:21.444000+00:00, 2021-12-29 22:03:21.486000+00:00, 2021-12-29 22:03:21.523000+00:00, 2021-12-29 22:03:21.534000+00:00, 2021-12-29 22:03:21.588000+00:00, 2021-12-29 22:03:21.614000+00:00, 2021-12-29 22:03:21.670000+00:00, 2021-12-29 22:03:21.697000+00:00, 2021-12-29 22:03:21.750000+00:00, 2021-12-29 22:03:21.812000+00:00, 2021-12-29 22:03:21.817000+00:00, 2021-12-29 22:03:21.890000+00:00, 2021-12-29 22:03:23.185000+00:00, 2021-12-29 22:03:...",158,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...]",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.9,1.0,1.0,1.0,1.0


In [22]:
df['tsLow'] = df['deltas'].apply(lambda x: np.percentile(np.array(x), 20))
df['tsMid'] = df['deltas'].apply(lambda x: np.percentile(np.array(x), 50))
df['tsHigh'] = df['deltas'].apply(lambda x: np.percentile(np.array(x), 80))
df['tsBowleyNum'] = df['tsLow'] + df['tsHigh'] - 2*df['tsMid']
df['tsBowleyDen'] = df['tsHigh'] - df['tsLow']
df['tsSkew'] = df[['tsLow','tsMid','tsHigh','tsBowleyNum','tsBowleyDen']].apply(
    lambda x: x['tsBowleyNum'] / x['tsBowleyDen'] if x['tsBowleyDen'] != 0 and x['tsMid'] != x['tsLow'] and x['tsMid'] != x['tsHigh'] else 0.0, axis=1
    )
df['tsMadm'] = df['deltas'].apply(lambda x: np.median(np.absolute(np.array(x) - np.median(np.array(x)))))
df['tsConnDiv'] = df[f_timestamp].apply(lambda x: (x[-1].to_pydatetime() - x[0].to_pydatetime()).seconds / 10)

In [23]:
df.head(3)

,source.ip,destination.ip,virtual_host,destination.port,http.method,ts,conn_count,deltas,tsLow,tsMid,tsHigh,tsBowleyNum,tsBowleyDen,tsSkew,tsMadm,tsConnDiv,tsSkewScore,tsMadmScore,tsConnCountScore,tsScore
0,192.168.1.154,8.240.196.126,download.windowsupdate.com,80,GET,"[2021-12-29 22:03:21.246000+00:00, 2021-12-29 22:03:21.305000+00:00, 2021-12-29 22:03:21.331000+00:00, 2021-12-29 22:03:21.405000+00:00, 2021-12-29 22:03:21.444000+00:00, 2021-12-29 22:03:21.486000+00:00, 2021-12-29 22:03:21.523000+00:00, 2021-12-29 22:03:21.568000+00:00, 2021-12-29 22:03:21.588000+00:00, 2021-12-29 22:03:21.615000+00:00, 2021-12-29 22:03:21.670000+00:00, 2021-12-29 22:03:21.697000+00:00, 2021-12-29 22:03:21.750000+00:00, 2021-12-29 22:03:21.812000+00:00, 2021-12-29 22:03:21.818000+00:00, 2021-12-29 22:03:21.890000+00:00, 2021-12-29 22:03:23.185000+00:00, 2021-12-29 22:03:...",158,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...]",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.9,1.0,1.0,1.0,1.0
1,192.168.128.50,8.240.196.126,download.windowsupdate.com,80,GET,"[2021-12-29 22:03:21.246000+00:00, 2021-12-29 22:03:21.305000+00:00, 2021-12-29 22:03:21.331000+00:00, 2021-12-29 22:03:21.405000+00:00, 2021-12-29 22:03:21.444000+00:00, 2021-12-29 22:03:21.486000+00:00, 2021-12-29 22:03:21.523000+00:00, 2021-12-29 22:03:21.534000+00:00, 2021-12-29 22:03:21.588000+00:00, 2021-12-29 22:03:21.614000+00:00, 2021-12-29 22:03:21.670000+00:00, 2021-12-29 22:03:21.697000+00:00, 2021-12-29 22:03:21.750000+00:00, 2021-12-29 22:03:21.812000+00:00, 2021-12-29 22:03:21.817000+00:00, 2021-12-29 22:03:21.890000+00:00, 2021-12-29 22:03:23.185000+00:00, 2021-12-29 22:03:...",158,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...]",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.9,1.0,1.0,1.0,1.0


In [24]:
df['tsSkewScore'] = 1.0 - abs(df['tsSkew'])
df['tsMadmScore'] = 1.0 - df['tsMadm']/30.0
df['tsMadmScore'] = df['tsMadmScore'].apply(lambda x: 0 if x < 0 else x)
df['tsConnCountScore'] = (df['conn_count']) / df['tsConnDiv']
df['tsConnCountScore'] = df['tsConnCountScore'].apply(lambda x: 1.0 if x > 1.0 else x)
df['tsScore'] = (((df['tsSkewScore'] + df['tsMadmScore'] + df['tsConnCountScore']) / 3.0) * 1000) / 1000
df.sort_values(by= 'tsScore', ascending=False, inplace=True, ignore_index=True)
df[columns_to_display].head(30)

,tsScore,conn_count,source.ip,destination.ip,virtual_host,http.method,destination.port,deltas
0,1.0,158,192.168.1.154,8.240.196.126,download.windowsupdate.com,GET,80,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...]"
1,1.0,158,192.168.128.50,8.240.196.126,download.windowsupdate.com,GET,80,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...]"


In [25]:
df.loc[df['tsScore'] > 0.80, columns_to_display]

,tsScore,conn_count,source.ip,destination.ip,virtual_host,http.method,destination.port,deltas
0,1.0,158,192.168.1.154,8.240.196.126,download.windowsupdate.com,GET,80,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...]"
1,1.0,158,192.168.128.50,8.240.196.126,download.windowsupdate.com,GET,80,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...]"
